# Real-Time E-commerce Data Pipeline with Spark ETL

## Project Overview

In this project, you will design and implement an ETL (Extract, Transform, Load) pipeline for a hypothetical e-commerce platform named **ShopEase**. The platform generates massive amounts of data daily, including user interactions, transactions, and inventory updates. Your task is to process this data using Apache Spark to derive meaningful insights and support real-time analytics.


## Scenario

**ShopEase** aims to enhance its data analytics capabilities to improve user experience, optimize inventory management, and increase sales. The data generated includes:

- **User Activity Logs:** Clickstream data capturing user interactions on the website.
- **Transaction Records:** Details of purchases, refunds, and returns.
- **Inventory Updates:** Information about stock levels, restocks, and product information changes.
- **Customer Feedback:** Reviews and ratings provided by customers.

The company requires a robust ETL pipeline to process this data, perform transformations, and make it available for analytics and reporting in both batch and real-time modes.


## Project Requirements

You are required to perform the following tasks using Apache Spark (preferably with PySpark or Scala):

### 1. Data Ingestion
- **Batch Data:**
  - Load historical data from large CSV and JSON files stored in your local file system.
- **Real-Time Data:**
  - Simulate and ingest streaming data from a Kafka topic representing live user activity logs.

### 2. Data Processing and Transformation
- **Using RDDs:**
  - Perform a transformation to filter out any corrupted or incomplete records from the transaction data.
  - Implement a custom transformation to anonymize user IDs for privacy compliance.
- **Using DataFrames:**
  - Clean and standardize inventory data (e.g., handling missing values, normalizing text).
  - Join user activity logs with transaction records to analyze user behavior leading to purchases.
- **Using Spark SQL:**
  - Create temporary views and execute SQL queries to compute:
    - Top 10 most purchased products in the last month.
    - Monthly revenue trends.
    - Inventory turnover rates.

### 3. Real-Time Streaming Processing (Optional but Recommended)
- Set up a Spark Streaming job to process incoming user activity logs.
- Compute real-time metrics such as:
  - Active users per minute.
  - Real-time conversion rates.
  - Detect and alert on unusual spikes in specific user activities.

### 4. Data Storage
- Store the transformed data into appropriate storage systems:
  - Use Parquet format for batch-processed data in a local data lake.
  - Use an in-memory data store like Redis or a database like PostgreSQL for real-time metrics.

### 5. Performance Optimization
- Optimize Spark jobs for better performance by:
  - Caching intermediate DataFrames where necessary.
  - Tuning Spark configurations (e.g., partition sizes, executor memory).
  - Using appropriate join strategies.

### 6. Documentation and Reporting
- Document the ETL pipeline architecture.
- Provide sample dashboards or reports (using Spark's built-in visualization) showcasing the insights derived.


## Questions & Requirements

### 1. Data Ingestion
- **Q1:** How did you ingest the batch and real-time data? Provide code snippets demonstrating the loading of data using RDDs and DataFrames.

### 2. Data Cleaning and Transformation
- **Q2:** Describe the transformations applied to the transaction data using RDDs. How did you ensure data quality and privacy?

### 3. DataFrame Operations
- **Q3:** How did you clean and standardize the inventory data using DataFrames? Provide examples of handling missing values and normalizing text fields.

### 4. Spark SQL Queries
- **Q4:** Present the Spark SQL queries used to calculate the top 10 most purchased products, monthly revenue trends, and inventory turnover rates.

### 5. Real-Time Processing
- **Q5:** If implemented, explain how the real-time streaming was set up. What metrics were computed in real-time, and how were they stored/displayed?

### 6. Performance Optimization
- **Q6:** What strategies did you employ to optimize the performance of your Spark jobs? Provide examples of configuration settings or code optimizations.

### 7. Reporting
- **Q7:** Show sample outputs or dashboards that visualize the insights derived from the ETL pipeline.


In [ ]:
from pyspark.sql import SparkSession
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("ShopEase_ETL") \
    .getOrCreate()

# Load transactions data (CSV format)
transactions_df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/content/large_transactions.csv")

# Load inventory data (JSON format)
inventory_df = spark.read.format("json").option("multiline", "true").load("/content/large_inventory.json")

# Load customer feedback data (JSON format)
feedback_df = spark.read.format("csv").option("header", "true") \
    .option("inferSchema", "true") .load("/content/large_customer_feedback.csv")

# Displaying a sample of each dataset
transactions_df.show(5)
feedback_df.show(5)
inventory_df.show(5)


In [ ]:
# Step 2: Data Cleaning and Transformation with RDDs
# Convert transactions DataFrame to RDD
transactions_rdd=transactions_df.rdd
cleaned_rdd = transactions_rdd.filter(lambda row: row.transaction_id is not None 
                                      and row.user_id is not None 
                                      and row.amount > 0)
cleaned_rdd.take(5)
# Write Function to Anonymize user IDs using Hashing
def anonymize(record):
    
import hashlib
# Function to anonymize user IDs
def anonymize_user_id(row):
    anonymized_user_id = hashlib.sha256(str(row.user_id).encode()).hexdigest()
    return (row.transaction_id, anonymized_user_id, row.product_id, row.quantity, row.amount, row.transaction_date)

anonymized_rdd = cleaned_rdd.map(anonymize_user_id)

# Convert back to DataFrame
anonymized_transactions_df = anonymized_rdd.toDF(["transaction_id", "user_id", "product_id", "quantity", "amount", "transaction_date"])

# Show anonymized data
anonymized_transactions_df.show(5)


In [ ]:
# Step 3: DataFrame Operations for Cleaning and Transformation
from pyspark.sql.functions import col, lower, trim

# Clean inventory data by handling missing values and normalizing text
cleaned_inventory_df = inventory_df.dropna(subset=["stock_level"]) \
                                  .withColumn("product_name", lower(trim(col("product_name"))))

# Display cleaned inventory data
cleaned_inventory_df.show(5)

# Perform a join operation to combine data
joined_df = anonymized_transactions_df.join(cleaned_inventory_df, on="product_id", how="inner")

# Display joined DataFrame
joined_df.show(5)

In [ ]:
# Step 4: Spark SQL Queries
# Create temporary views for SQL queries
anonymized_transactions_df.createOrReplaceTempView("transactions")
cleaned_inventory_df.createOrReplaceTempView("inventory")
joined_df.createOrReplaceTempView("joined_data")


# Query: Monthly revenue trends
monthly_revenue_df = spark.sql("""
    SELECT MONTH(transaction_date) AS month, YEAR(transaction_date) AS year, 
           SUM(amount * price) AS monthly_revenue
    FROM joined_data
    GROUP BY month, year
    ORDER BY year, month
""")
monthly_revenue_df.show()

# Query: Top 10 most purchased products in the last month
top_products_df = spark.sql("""
    SELECT product_name, SUM(amount) AS total_amount
    FROM joined_data
    WHERE transaction_date >= '2023-08-01' AND transaction_date <= '2023-08-31'
    GROUP BY product_name
    ORDER BY total_amount DESC
    LIMIT 10
""")
top_products_df.show()

# Query: Inventory turnover rates (Total Sales per Product)
turnover_rate_df = spark.sql("""
    SELECT product_name, SUM(amount * price) AS turnover_rate
    FROM joined_data
    GROUP BY product_name
    ORDER BY turnover_rate DESC
    LIMIT 20
""")
turnover_rate_df.show()


In [ ]:
# Step 5: Real-Time Processing (Optional)
from pyspark.sql.functions import window, countDistinct

# For demonstration, create a streaming DataFrame from a sample batch dataset
streaming_transactions_df = spark.readStream.schema(transactions_df.schema) \
                                           .csv("streaming_transactions_folder")  # Point to a folder with incoming data

# Compute real-time metrics (e.g., active users per minute)
active_users = 

# Display active users in real-time (Note: This will print continuously if run with actual streaming data)
query = 

query.awaitTermination()  # Keep the stream running (can be stopped manually)

In [ ]:
# Step 6: Performance Optimization Techniques

# Caching DataFrames to optimize performance for multiple transformations
anonymized_transactions_df.cache()
cleaned_inventory_df.cache()
joined_df.cache()

# Repartition DataFrames to optimize join performance
# Choose an optimal number of partitions based on your data size
transactions_df_repartitioned = anonymized_transactions_df.repartition(10, "product_id")
inventory_df_repartitioned = cleaned_inventory_df.repartition(10, "product_id")

# Use Broadcast Join for small DataFrames (if applicable)
# Assuming inventory_df is relatively small compared to transactions_df
from pyspark.sql.functions import broadcast
joined_df_optimized = transactions_df_repartitioned.join(broadcast(inventory_df_repartitioned), "product_id")

# Display the optimized joined DataFrame
joined_df_optimized.show(5)


In [ ]:
# Step 7: Store the Transformed Data

# Store the cleaned and transformed data in Parquet format
anonymized_transactions_df.write.mode("overwrite").parquet("file:///home/student/mini_project/anonymized_transactions.parquet")
cleaned_inventory_df.write.mode("overwrite").parquet("file:///home/student/mini_project/cleaned_inventory.parquet")
joined_df_optimized.write.mode("overwrite").parquet("file:///home/student/mini_project/joined_data.parquet")


## Sample Dashboard Outputs
- **Top Products Bar Chart:** Displaying the top 10 products with the highest sales.
- **Revenue Trend Line Chart:** Showing monthly revenue over the past year.
- **Inventory Turnover Heatmap:** Visualizing turnover rates across different product categories.

*(Include actual screenshots or detailed descriptions as appropriate.)*


#### 1. Top Products Bar Chart - Displaying the top 10 products with the highest sales

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming top_products_df is your DataFrame
top_products = top_products_df.toPandas()

plt.figure(figsize=(12, 8))  # Adjusted size for better readability
plt.bar(top_products['product_name'], top_products['total_amount'], color='skyblue')
plt.title("Top 10 Products by Sales")
plt.xlabel("Product Name")
plt.ylabel("Total Sales Amount")
plt.xticks(rotation=90)  # Rotate x-axis labels to be horizontal for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Added grid for y-axis for better readability
plt.tight_layout()  # Adjust layout to fit labels and titles
plt.show()


#### 2. Revenue Trend Line Chart - Showing monthly revenue over the past yea

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming monthly_revenue_df is your DataFrame
monthly_revenue = monthly_revenue_df.toPandas()

plt.figure(figsize=(12, 8))  # Adjust size for better readability
plt.plot(monthly_revenue['month'], monthly_revenue['monthly_revenue'], marker='o', color='green', linestyle='-', linewidth=2)
plt.title("Monthly Revenue Trend in 2023")
plt.xlabel("Month")
plt.ylabel("Revenue")

# Ensure that 'month' column is in a format suitable for plotting
# For example, if 'month' is a string, it should be sorted and formatted appropriately
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True, linestyle='--', alpha=0.7)  # Grid for better readability
plt.tight_layout()  # Adjust layout to fit labels and titles
plt.show()


#### 3. Inventory Turnover Heatmap - Visualizing turnover rates across different product categories
Assuming you have a product_category column in the inventory data.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming joined_df_optimized is your DataFrame with product_category
inventory_turnover = inventory_df_repartitioned.select("product_name", "product_category", "amount", "price") \
    .groupBy("product_category", "product_name") \
    .agg({"amount": "sum", "price": "mean"}) \
    .withColumn("turnover", col("sum(amount)") * col("avg(price)")) \
    .toPandas()

# Pivot the data for the heatmap
heatmap_data = inventory_turnover.pivot(index="product_category", columns="product_name", values="turnover")

# Create the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap="viridis", annot=True, fmt=".2f")
plt.title("Inventory Turnover Heatmap")
plt.xlabel("Product Name")
plt.ylabel("Product Category")
plt.show()

## Submission Guidelines
- Ensure that your code is well-documented and follows best practices.
- Include instructions on how to set up and run your ETL pipeline.
- Provide all necessary configurations and dependencies required for execution.
- If using external services (like Kafka or Redis), include setup instructions or mock implementations.


## Additional Resources
- [Apache Spark Documentation](https://spark.apache.org/documentation.html)
- [Spark Structured Streaming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Optimizing Spark Performance](https://spark.apache.org/docs/latest/tuning.html)
- [Using Spark SQL](https://spark.apache.org/docs/latest/sql-programming-guide.html)
